# Perceptron

## Gradient

Premièrement, amusons nous un peu avec des maths. 

Pour appliquer la descente de gradient, on a évidemment besoin de calculer le gradient de notre fonction de coût (ou perte). Supposons le perceptron suivant avec un rectifieur comme couche d'activation.

$y = w^Tx+b$

$a = rect(y)$ où $rect(x) = \left\{ 
  \begin{array}{l l}
    0 & \quad \text{si $x\le0$}\\
    x & \quad \text{sinon}
  \end{array} \right.$
  
Normalement on utiliserait un softmax pour avoir un vecteur de probabilité, mais j'utilise ici un exemple bidon pour vous faire pratiquer quelques principes de dérivation.

Supposons maintenant qu'on utilise comme fonction de coût la différence au carrée, soit:

$L(a,t) = (a-t)^2$

Si on remplace les variables on obtient 

$\displaystyle L(a,t) = (rect(w^Tx+b)-t)^2$

Ce qui est encore supportable pour les yeux, mais comme vous avez pu le voir dans le devoir, ça peut vite dégénérer. L'idéal, lorsqu'on dérive notre fonction de coût, c'est de tirer profit de ce qu'on appelle la dérivation en chaîne. 



### Rappel sur les dérivations

Petit rappel: La règle de dérivation en chaîne nous dit que 

$\displaystyle \frac{\partial y}{\partial x} = \frac{\partial y}{\partial u}\frac{\partial u}{\partial x}$ ($y$, $x$ et $u$ n'ont rien à voir avec nos formules.)

Alors si on applique la règle à notre problème, on obtient :

$\displaystyle \frac{\partial L(a,t)}{\partial w} = \frac{\partial L(a,t)}{\partial a}\frac{\partial a}{\partial y}\frac{\partial y}{\partial w}$

Autrement dit, pas besoin de se battre (et de possiblement faire 1001 erreurs de calculs) avec la formule complète de $\frac{\partial L(a,t)}{\partial w}$, on peut calculer séparément $\frac{\partial L(a,t)}{\partial a}$, $\frac{\partial a}{\partial y}$ et $\frac{\partial y}{\partial w}$. 

Rappelez vous de cette règle, elle vous sera nécessaire plus tard (pas aujourd'hui) pour la back-propagation.

#### Dérivation de w^Tx

Lorsqu'on dérive une opération vectorielle ou matricielle tel que $w^Tx$, il est toujours mieux de développer l'équation. Dans notre cas,

$\displaystyle w^Tx = \sum_{i=1}^{d} w_ix_i$

Ainsi, la dérivé n'est pas très compliqué.

$\displaystyle \frac{\partial}{\partial w}w^Tx = \frac{\partial}{\partial w}\sum_{i=1}^{d} w_ix_i = \sum_{i=1}^{d} \frac{\partial}{\partial w} w_ix_i$

Mais attention! $\frac{\partial}{\partial w}w_ix_i \neq x_i$, car on a bien $\frac{\partial}{\partial w}$ et non pas $\frac{\partial}{\partial w_i}$. Il faut donc y aller au cas par cas et calculer pour chaque dimension. 

$\displaystyle \left(\frac{\partial}{\partial w_1}w^Tx,\frac{\partial}{\partial w_2}w^Tx,\dots,\frac{\partial}{\partial w_n}w^Tx\right) = \left(\sum_{i=1}^d \frac{\partial}{\partial w_1} w_ix_i,\sum_{i=1}^d \frac{\partial}{\partial w_2} w_ix_i,\dots,\sum_{i=1}^d \frac{\partial}{\partial w_n} w_ix_i\right)$

Autrement dit 

$\displaystyle \frac{\partial}{\partial w_i}w^Tx = \sum_{j=1}^d \frac{\partial}{\partial w_i} w_jx_j$

Bon, j'en ai trop dit, à vous de terminer la dérivation.

### Dérivation particulière

Il arrive parfois qu'on se retrouve face à une fonction qui semble impossible à dériver, comme par exemple la fonction indicatrice ou $I_{qqc}$ ou bien la fonction $rect(x)$. Une manière simple de règler le problème est de «casser» l'équation en deux. 

Exemple: $rect(y)$

On sait que $rect(x) = \left\{ 
  \begin{array}{l l}
    0 & \quad \text{si $x\le0$}\\
    x & \quad \text{sinon}
  \end{array} \right.$
  
Si on remplace $x$ par l'équation $wx+b$, on obtient

$rect(wx+b) = \left\{ 
  \begin{array}{l l}
    0 & \quad \text{si $w^Tx+b\le0$}\\
    wx+b & \quad \text{sinon}
  \end{array} \right.$

Il ne reste plus qu'à remonter d'un niveau et on obtient 

$\displaystyle L(y,t) = \left\{ 
  \begin{array}{l l}
    t^2 & \quad \text{si $w^Tx+b\le0$}\\
    (w^Tx+b-t)^2 & \quad \text{sinon}
  \end{array} \right.$

On peut maintenant dériver $t^2$ et $(w^Tx+b-t)^2$. N'oubliez pas la dérivation en chaîne. La deuxième équation peut être écrite $(y-t)^2$.

Maintenant, à vos crayons, prêts, dérivez $\displaystyle\frac{\partial L(a,t)}{\partial w}$!

## Transformations non-linéaires

Pour démontrer l'utilité des transformations non-linéaires, nous allons utiliser les ensembles de données cercle et ellipse, disponible sur la page du site. On va premièrement appliquer une transformation sur les données avant l'entraînement. L'algorithme ne touchera que les données transformées. On ne se cassera pas la tête pour commencer, implémentez simplement une transformation polynomiale de deuxième degré.

Petit rappel:

$\phi_{\text{poly}^2}(x) = \left(x_1,x_2,\dots,x_d,
\alpha_{11} x_1^2, \alpha_{22}x2^2, \dots, \alpha_{dd}x_d^2,
\alpha_{12}x_1x_2, \alpha_{13}x_1x_2, \dots, \alpha_{1d}x_1x_d, \dots, \alpha_{(d-1)d}x_{d-1}x_d\right)$

Pour faire simple, on peut mettre $\alpha_i=1$ $\forall i$

Et pour faire encore plus simple, les données cercles et ellipse sont seulement en 2-d...

In [5]:
%pylab inline
import numpy as np
import utilitaires
import time

Populating the interactive namespace from numpy and matplotlib


In [6]:
# prends une matrice de données en entrée (sans cibles) et renvoie une matrice transformée avec une polynomiale de degrée p

def polynome(x,p=2):
    new_elements = x**2 # À vous d'implémenter les nouveaux éléments (ça peut très bien pres plus d'une ligne de code)
    new_x = np.concatenate((x,new_elements),axis=1)
    return new_x

In [7]:
class perceptron:
	def __init__(self, mu):
		self.mu = mu
    
	def train(self, train_data):
          nb_example = train_data.shape[0]

          self.weights = np.random.random(train_data.shape[1])
          self.weights[-1] = 0
          datas = np.array(train_data)
          datas[:,-1] = 1
          i = 0
          count = 0 # We stop when the set is linearly separated
          n_iter = 0
          n_iter_max = nb_example*100
          while (count < nb_example and n_iter < n_iter_max):
            if (np.dot(datas[i], self.weights)) * train_data[i,-1] < 0:
              self.weights += self.mu * train_data[i,-1] * datas[i]
              count = 0
            else:
              count = count + 1
            i = (i + 1) % nb_example
            n_iter += 1

	def compute_predictions(self, test_data):

           sorties = []
           for i in range(len(test_data)):
             data = []
             for j in range(len(test_data[i])):
               data.append(test_data[i][j])
             data.append(1)
             sorties.append(np.dot(data, self.weights))
           return sorties

In [8]:
# On commence par charger les données
data = np.loadtxt('cercle.txt')
#data = np.loadtxt('ellipse.txt')

# Il n'y a que deux dimensions...
train_cols = [0,1]
# Une variable pour contenir l'indice de la colonne correspondant aux étiquettes.
target_ind = [data.shape[1] - 1]

# Nombre de classes
n_classes = 2
# Nombre de points d'entrainement
n_train = 1500
# Taille de la grille = grid_size x grid_size
grid_size = 50

print "On va entrainer un perceptron sur ", n_train, " exemples d'entrainement"

# decommenter pour avoir des resultats non-deterministes 
random.seed(3395)

# Déterminer au hasard des indices pour les exemples d'entrainement et de test
inds = range(data.shape[0])
random.shuffle(inds)
train_inds = inds[:n_train]
test_inds = inds[n_train:]
    
# Separer les donnees dans les deux ensembles: entrainement et test.
train_set = data[train_inds,:]	# garder les bonnes lignes
train_set = train_set[:,train_cols + target_ind]  # garder les bonnes colonnes
test_set = data[test_inds,:]
test_set = test_set[:,train_cols + target_ind]

# Separarer l'ensemble de test: entrees et etiquettes.
test_inputs = test_set[:,:-1]
test_labels = test_set[:,-1]

# Le taux d'apprentissage
mu = 0.005

# Transforme les données
transformed_train_set = np.concatenate((polynome(train_set[:,:-1]),train_set[:,-1][:,None]),axis=1)
transformed_test_inputs = polynome(test_inputs)

# Créer et entrainer le modele
model_perceptron = perceptron(mu)
model_perceptron.train(transformed_train_set)

# Obtenir les sorties sur l'ensemble de test.
t1 = time.clock()
les_sorties = model_perceptron.compute_predictions(transformed_test_inputs)
t2 = time.clock()
print 'Ca nous a pris ', t2-t1, ' secondes pour calculer les predictions sur ', transformed_test_inputs.shape[0],' points de test'

# Convertir les sorties en classe. On prend le signe.
classes_pred = np.sign(les_sorties)
   
# Mesurer la performance.
err = 1.0 - np.mean(test_labels==classes_pred)
print "L'erreur de test est de ", 100.0 * err,"%"

# Affichage graphique
if len(train_cols) == 2:
    # Surface de decision
    t1 = time.clock()
    les_sorties = model_perceptron.compute_predictions(transformed_train_set[:,:-1])
    # Convertir les sorties en classe. On prend le signe.
    train_classes_pred = np.sign(les_sorties)
    plt.scatter(train_set[:,0],train_set[:,1],c=train_classes_pred)
    plt.scatter(test_set[:,0],test_set[:,1],c=classes_pred)
    plt.show()
    
    plt.scatter(train_set[:,0],train_set[:,1],c=train_set[:,-1])
    plt.scatter(test_set[:,0],test_set[:,1],c=test_labels)
    plt.show()

    t2 = time.clock()
    print('Ca nous a pris ', t2-t1, ' secondes pour calculer les predictions sur ', grid_size * grid_size, ' points de la grille')
    filename = 'grille_' + '_c1=' + str(train_cols[0]) + '_c2=' + str(train_cols[1])+'.png'
    print('On va sauvegarder la figure dans ', filename)
    pylab.savefig(filename,format='png')
        
else:
    print('Trop de dimensions (', len(train_cols),') pour pouvoir afficher la surface de decision')

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-8-029a2407634d>, line 17)

## Astuce du noyau

Votre objectif pour cette section sera d'implanter un perceptron à noyau gaussien. On vous demande d'utiliser un kernel gaussien pour ce problème. Pour revoir les bases, référez-vous au [document du cours](https://studium.umontreal.ca/mod/resource/view.php?id=416954), notamment la deuxième partie. Vous pouvez vous aider avec le pseudo-code [ici](http://cvit.iiit.ac.in/thesis/ranjeethMS2007/thesis/node21.html).

Commencez par implémenter une fonction de kernel gaussien qui pourra vous servir pour faire le calcul initial. Il ne devrait pas être nécessaire de rappeler la fonction à chaque époque de l'entraînement.

In [ ]:
def kernel(?):
    k = np.zeros((?,?))
    return k

Vous pouvez ensuite compléter le code suivant.

In [ ]:
class KernelPerceptron:
    def __init__(self, mu):
        self.mu = mu

    def train(self, train_data, kernel_function):
        nb_example = train_data.shape[0]
        
        # Initialise les alphas
        
        # Retire les cibles
        datas = np.array(train_data)[:,:-1]
        
        # Calcul du kernel
    
        i = 0
        count = 0 
        n_iter = 0
        n_iter_max = nb_example*100
        while (count < nb_example and n_iter < n_iter_max):
            # comment calculer la prédiction?
            if ?*train_data[i,-1] < 0:
                self.alphas += # comment alpha est mis à jour?
                count = 0
            else:
                count = count + 1
            i = (i + 1) % nb_example
            n_iter += 1

    def compute_predictions(self, test_data, kernel_function):
        # Il faut calculer le kernel
        
        sorties = []
        for i in range(len(test_data)):
            data = []
            for j in range(len(test_data[i])):
               data.append(test_data[i][j])
            data.append(1)
            
            # comment calculer la prédiction?
            sorties.append(?)
        return sorties

In [ ]:
# charger les donnees
#data = np.loadtxt('ellipse.txt')
data = np.loadtxt('cercle.txt')
type_transformation=2

# Les colonnes (traits/caracteristiques) sur lesqueles on va entrainer notre modele
# Pour que gridplot fonctionne, len(train_cols) devrait etre 2
train_cols = [0,1]
# L'indice de la colonne contenant les etiquettes
target_ind = [data.shape[1] - 1]

# Nombre de classes
n_classes = 2
# Nombre de points d'entrainement
n_train = 1500
# Taille de la grille = grid_size x grid_size
grid_size = 50

print "On va entrainer un algo lineaire sur ", n_train, " exemples d'entrainement"

# decommenter pour avoir des resultats non-deterministes 
random.seed(3395)
# DÃ©terminer au hasard des indices pour les exemples d'entrainement et de test
inds = range(data.shape[0])
random.shuffle(inds)
train_inds = inds[:n_train]
test_inds = inds[n_train:]

# separer les donnees dans les deux ensembles
train_set = data[train_inds,:]
train_set = train_set[:,train_cols + target_ind]
test_set = data[test_inds,:]
test_set = test_set[:,train_cols + target_ind]

# separarer l'ensemble de test dans les entrees et les etiquettes
test_inputs = test_set[:,:-1]
test_labels = test_set[:,-1]

mu = 0.00005
model = KernelPerceptron(mu)
model.train(train_set, kernel)

# Obtenir ses prÃ©dictions
t1 = time.clock()
les_sorties = model.compute_predictions(test_inputs, kernel)

t2 = time.clock()
print 'Ca nous a pris ', t2-t1, ' secondes pour calculer les predictions sur ', test_inputs.shape[0],' points de test'

# Vote majoritaire (+1 puisquie nos classes sont de 1 a n)
classes_pred = np.sign(les_sorties)

# Faire les tests
err = 1.0 - np.mean(test_labels==classes_pred)
print "L'erreur de test est de ", 100.0 * err,"%"

if len(train_cols) == 2:
    # Surface de decision
    t1 = time.clock()
    les_sorties = model_perceptron.compute_predictions(train_set[:,:-1],kernel)
    train_classes_pred = np.sign(les_sorties)
    plt.scatter(train_set[:,0],train_set[:,1],c=train_classes_pred)
    plt.scatter(test_set[:,0],test_set[:,1],c=classes_pred)
    plt.show()
    
    plt.scatter(train_set[:,0],train_set[:,1],c=train_set[:,-1])
    plt.scatter(test_set[:,0],test_set[:,1],c=test_labels)
    plt.show()
    t2 = time.clock()
    print 'Ca nous a pris ', t2-t1, ' secondes pour calculer les predictions sur ', grid_size * grid_size, ' points de la grille'
    filename = 'grille_' + '_c1=' + str(train_cols[0]) + '_c2=' + str(train_cols[1])+'.png'
    print 'On va sauvegarder la figure dans ', filename
    pylab.savefig(filename,format='png')
else:
    print 'Trop de dimensions (', len(train_cols),') pour pouvoir afficher la surface de decision'